# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# Импортируем библиотеки

import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [2]:
# Создаем датафрейм с исходными данными
# Посмотрим что из себя представляют данные

df = pd.read_csv('/datasets/data.csv')
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# Посмотрим общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итого: В данных 12 столбцов. Есть количественный и категориальные значения, в двух столбцах есть пропуски. Посмотрим поближе по очереди

## Найдем и обработаем возможные ошибки

In [4]:
# Посмотрим значения нечисловых столбцов

print(df['education'].unique())
print(df['family_status'].unique())
print(df['gender'].unique())
print(df['income_type'].unique())
print(df['purpose'].unique())

# Нужно привести к одному формату уровень образования
df['education'] = df['education'].str.lower()
df['education'].unique()

# Посмотрим на неопределенный пол
df.loc[(df['gender'] == 'XNA')]

# Всего одна строка - удалим ее
df = df.drop(df[df['gender'] == 'XNA'].index)
df['gender'].unique()

# Посмотрим тип занятости
df.groupby('income_type').count()
# В основном данные по госслужащим, компаньонам, пенсионерам и сотрудникам
# По остальным типам слишком мало информации, что бы делать какие-то выводы. Лучше убирем, что бы не мешались
df = df.drop(df[(df['income_type'] == 'безработный') | (df['income_type'] == 'в декрете') | (df['income_type'] == 'предприниматель')| (df['income_type'] == 'студент')].index)
df['income_type'].unique()

# В целях много повторений - нужно будет поделить их на категории

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образо

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий'], dtype=object)

In [5]:
# Описательная статистика по числовым данным  
print(df.describe())  
print(df['education_id'].unique())
print(df['family_status_id'].unique())
print(df['debt'].unique())

           children  days_employed     dob_years  education_id  \
count  21518.000000   19345.000000  21518.000000  21518.000000   
mean       0.538944   63028.519199     43.296031      0.817316   
std        1.381748  140811.982962     12.573894      0.548052   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.548926     33.000000      1.000000   
50%        0.000000   -1203.506819     42.000000      1.000000   
75%        1.000000    -291.132779     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  
count      21518.000000  21518.000000  1.934500e+04  
mean           0.972535      0.080816  1.674164e+05  
std            1.420358      0.272559  1.029519e+05  
min            0.000000      0.000000  2.066726e+04  
25%            0.000000      0.000000  1.030824e+05  
50%            0.000000      0.000000  1.450179e+05  
75%            1.000000    

Столбцы education_id, family_status_id, debt (тип int64) - содержат данные о категориях. Проверили уникальные значения, никаких аномалий нет. 
Смотрим на оставшиеся столбы. total_income - ежемесячный доход - отрицательных значений нет, проверить корректность нет возможности
Столбец children - дети - есть отрицательные значение - явно ошибка; максимальное значение 20 - странно, физически это возможно, но маловероятно, надо проверить
Столбец days_employed - дни стажа. Отрицательные значения - ошибки. Также есть слишком большие значения, физически невозможные, ну и посмотрим соответствие возраста и стажа
Столбец dob_years - возраст. Минимальный возраст не может быть 0
Итого в трех столбцах есть ошибки

In [6]:
# Начнем с исправления ошибок children

df.loc[(df['children'] == 20)]
df.loc[(df['children'] == 20)].count()
df.loc[(df['children'] > 5)].count()

# Посмотрим, а есть ли вообще люди с числом детей больше 5. Видим, что это те же строки.
# Нет не одного человека с количеством детей от 6 до 19. Значит 20 это ошибка.
# Либо это лишний ноль, либо какая-то техническая ошибка.Можно было бы заменить все значения 20 на 2.
# Но если в системе возможна ошибка лишнего нуля, почему нет ошибок с количеством детей 10,30,40,50?
# В целом таких строк не так много, можно избавиться от ошибок удалив их

df = df.drop(df[df['children'] == 20].index)

# Посмотрим на отрицательное количество детей
df.loc[(df['children'] < 0)]
# # Еще строки с -1 ребенком. Аналогично почему нет -2, -3?? Пожертвуем и этими строками. Их не так много

df = df.drop(df[df['children'] == -1].index)

# # Посмотрим, что в итоге все корректно

df['children'].describe()


count    21395.000000
mean         0.473195
std          0.751844
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

In [7]:
# Переходим к возрасту

df.loc[(df['dob_years'] == 0)]

# Видим 100 строк с нулевым значением возраста
# Что бы не удалять еще больше строк попробуем заменить средним возрастом по группе

df['dob_years'] = df.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))

df['dob_years'].describe()

# Нулевых значений не осталось

count    21395.000000
mean        43.504136
std         12.235494
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [8]:
# Найдем ошибки в стаже

df.loc[(df['days_employed'] < 0)]

# 15805 строк с отрицательных значение - больше половины выборки. Вероятно, какая-то общая техническая ошибка. 
# Причину этой ошибки нужно найти и исправить. Пока же делаем предположение, что в записях просто лишний минус. Убирем его

df['days_employed'] = df['days_employed'].abs()


# Теперь посмотрим максимальные значение. Для этого создадим вспомогательный столбец. 
# В него запишем возможен ли такой стаж для каждого человека (исходя из факта, что работать можно с 14 лет)
# Таким образом, отрицательные значения в этом столбце - ошибки

df['experience'] = df['dob_years'] - 14 - df['days_employed']/365
df.loc[(df['experience'] < 0)]
df.loc[(df['experience'] < 0)].groupby('income_type').count()

# Большее число ошибок в категории пенсионеров.
# Заменим стаж разделив его на условный коэффициент исходя из возраста
# Предположим, что человек работает с 14 лет. Если от возраста отнять 14, то получим максимально возможный стаж человека
# Разделим указанный стаж на максимально возможный и найдем среднее


df['coef'] = (df['days_employed']/365)/(df['dob_years'] - 14)
coef = df.loc[(df['experience'] < 0)].coef.mean()

# Теперь поделим каждое значение на полученный коэффициент

df.loc[(df['experience'] < 0),'days_employed'] = df['days_employed']/coef

# Не идеально, но для целей анализа подойдет. Стаж у пенсионеров выше, чем у остальных групп, зашкаливающих значений нет
df.groupby('income_type').mean()

# Удалим лишнее и посмотрим, что вышло

df = df.drop(['experience','coef'], axis=1)
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21395.000000,19234.000000,21395.000000,21395.000000,21395.000000,21395.000000,1.923400e+04
mean,0.473195,4821.144597,43.504136,0.817200,0.973125,0.080860,1.674429e+05
std,0.751844,5725.932250,12.235494,0.548655,1.420571,0.272626,1.030854e+05
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,926.191736,34.000000,1.000000,0.000000,0.000000,1.030109e+05
50%,0.000000,2189.232143,43.000000,1.000000,0.000000,0.000000,1.450208e+05
75%,1.000000,5521.612902,53.000000,1.000000,1.000000,0.000000,2.034396e+05
max,5.000000,17888.607457,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Данные содержат 21525 строк.  
Сразу видно пропуски в столбцах days_employed(трудовой стаж) и total_income(ежемесячный доход), причем пропусков одинаковое количество. Вероятно, это одни и теже строки. Люди не указавшие свой стаж не указывают доход, возможно есть стаж не указан, то в базу невозможно внести доход, так как для того что бы был доход, нужен хоть какой-то стаж.
Исправлены ошибки                                     1) отрицательное количество детей; 
                                                      2) отрицательное число дней стажа;
                                                      3) слишком большое количество дней стажа;
                                                      4) слишком маленький возраст.
Дальнейшие необходимые шаги: 1) Заполнить пропуски
                             2) Удалить дубликаты
                             3) Поделить цели кредита на категории

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
# Пропуски были найдены в двух столбцах. Посмотрим поближе. Начнем с days_employed
null_in_df = df[df['days_employed'].isnull()].count()
percent_null = null_in_df[0]/df.shape[0]*100
print('Процент пропусков', percent_null)

# Посмотрим на эти пропуски, что у них общего, есть ли закономерности
df_null = df[df['days_employed'].isnull()]
# Предположим логически, что пропуски в стаже связаны с типом занятости.
df_null.groupby('income_type').count()

Процент пропусков 10.100490768871232


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,146,0,146,146,146,146,146,146,146,0,146
компаньон,504,0,504,504,504,504,504,504,504,0,504
пенсионер,410,0,410,410,410,410,410,410,410,0,410
сотрудник,1101,0,1101,1101,1101,1101,1101,1101,1101,0,1101


In [10]:
# Нужно посмотреть какое распределение по типу занятости в исходных данных
df.groupby('income_type').count()

# Можем сравнить, что по каждому типу занятости в процентном отношении примерно одинаковое количество пропусков.
# Возможно пропуски связаны с какими-либо техническими проблемами
# Попробуем заменить пропуски 
# Заменять будем средним значением по группе 

df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('mean'))
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('mean'))

df.isnull().sum()

# Пропусков не осталось

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

 
В исходном датафрейме большое число пропусков. Иы избавились от них путем заполнения срднем значением

### Замена типа данных

In [11]:
# Заменим типы данных в тем столбцах, где это нужно

df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Количество дней стажа и доход привели к целочисленному виду, дробные части в этих данных излишни и ничего не дают. Теперь все данные выглядят гораздо приятнее и с ними удобнее работать

### Обработка дубликатов

In [12]:
# Проверим данные на дубликаты
df.duplicated().sum()
# В данных 71 строка с дубликатами. Выведем их
df[(df.duplicated(keep=False) == 1)].sort_values(by=['days_employed','dob_years']).head(15)
# Удалим строки и проверим, что дубликатов не осталось
df = df.drop_duplicates()
df.duplicated().sum()

0

**Вывод**

Повторяющиеся значения, вероятно, просто связаны с человеческим фактором. Всего дубликатов не очень много. Просто удалили их

### Лемматизация

In [13]:
# Необходимо привести цели кредита к одному виду, так как сейчас в целях есть разные формулировки одного и то же смысла
df_for_lemms = df.groupby('purpose')['purpose'].count()

# Для начала получим список всех возможных значений в столбце purpose
list_for_lemms = df['purpose'].tolist()


In [14]:
# Получим список всех возможных лем. Для этого создадим в список и пройдясь по всем значениям list_for_lemms
# Добавим леммы в список
lemms = []
for text in list_for_lemms:
    lem = m.lemmatize(text)
    for word in lem:
        lemms.append(word)

In [15]:
# Посчитаем число каждой леммы 
sorted(Counter(lemms).items(), key=lambda x: x[1])
# Отбираем значения. Список небольшой, очевидно можно выбрать категории логически - 
# Самые популярные варианты 'недвижимость','покупка','жилье','автомобиль','образование','операция','свадьба'
# Опускаем знаки, предлоги, местоимения. Из оставшегося убираем покупка' и 'операция', 
# так как эти слова могут быть связаны с разными целями

[('профильный', 432),
 ('приобретение', 459),
 ('подержать', 471),
 ('подержанный', 481),
 ('ремонт', 603),
 ('со', 623),
 ('собственный', 627),
 ('семья', 637),
 ('сдача', 648),
 ('сыграть', 760),
 ('проведение', 762),
 ('дополнительный', 899),
 ('заниматься', 904),
 ('сделка', 936),
 ('жилой', 1221),
 ('для', 1285),
 ('коммерческий', 1303),
 ('получение', 1309),
 ('высокий', 1367),
 ('строительство', 1866),
 ('на', 2206),
 ('свой', 2219),
 ('свадьба', 2312),
 ('операция', 2587),
 ('с', 2900),
 ('образование', 3988),
 ('автомобиль', 4278),
 ('жилье', 4435),
 ('покупка', 5862),
 ('недвижимость', 6311),
 ('\n', 21324),
 (' ', 33367)]

In [16]:
# Создадим функцию для определения категории в соответсвии с тем, какие выводы мы сделали на предыдущем шаге
def purpose_category(row):
    purpose_str = row['purpose']
    lem = m.lemmatize(purpose_str)
    if 'недвижимость' in lem:
        return 'недвижимость'
    elif 'жилье' in lem:
        return 'недвижимость'
    elif 'образование' in lem:
        return 'образование'
    elif 'автомобиль' in lem:
        return 'автомобиль'
    elif 'свадьба' in lem:
        return 'свадьба'
    else:
        return 'цель не определена'

In [17]:
# Применим функцию для создания нового столбца и убедимся, что она сработала корректно
df['purpose_category'] = df.apply(purpose_category, axis = 1)
df.head()
# Посмотрим, не упустили ли мы что-нибудь важное
df.loc[(df['purpose_category'] == 'цель не определена')]
# Строк с таким значением нет - значит все определено корректно


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category


**Вывод**

Разные люди разными словами описывают одни и те же цели, поэтому изначально в столбце purpose было множество уникальных значений. Приведение их всех к одному виду значительно упрощает дальнейшую работу с данными и позволяет удобнее групировать данные и делать выводы

### Категоризация данных

В исходных данных можно выделить различные категории людей по доходу, возрасту, числу детей и т.д. 
Для проведения категоризации выберем то, что понадобиться в дальнейшем при ответе на вопросы. В вопросах нужно определить влияет ли на возврат кредита в срок - 1) наличие детей; 2) семейное положение; 3) уровень дохода; 4) цели
Наличие детей от 1 до 5 итак хорошо и наглядно сгруппируется, семейное положение и цели фактически прокатегоризированы, 
а вот с доход все не так очевидно. Поэтому категоризацию проведем по уровню дохода

In [18]:
# Еще раз посмотрим

df['total_income'].describe()

# Для начала нужно определиться на сколько категорий делить. 
# Поделим на пять групп (аналогично с тем как выделены категории по семейному положению и образованию), 
# так как это будет достаточно и удобно для дальнейшего анализа 


df['total_income'].max() - df['total_income'].min() 
df['total_income'].mean()
df['total_income'].median()
# Средний доход 167 тыс, медианный чуть ниже - 150; 
# Разрыв между доходами более 2 миллиона, есть смысл брать крупные интервалы

151902.0

In [19]:
def income_category(row):
    income = row['total_income']
    if income < 50000:
        return 'низкий доход'
    elif income < 150000 :
        return 'ниже среднего'
    elif income < 200000:
        return 'средний доход'
    elif income < 500000:
        return 'богатые'
    else:
        return 'самые богатые'

In [20]:
# Применим функцию для создания нового столбца и убедимся, что она сработала корректно
df['income_category'] = df.apply(income_category, axis = 1)
df.head(20)
# Посмотрим, не упустили ли мы что-нибудь важное
# df.loc[(df['purpose_category'] == 'цель не определена')]
# Строк с таким значением нет - значит все определено корректно

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,богатые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,ниже среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,богатые
4,0,15150,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,богатые
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,богатые
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,ниже среднего
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,ниже среднего


**Вывод**

В данных большой разброс по уровню дохода, разбиение на группы позволит обобщить категории и в дальнейшем делать выводы

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
# Посмотрим на зависимость с помощью сводной таблицы. 
# Выводим количество клиентов, количество должников(sum, так как должник - это 1) и средний показатель по задолжности

df.pivot_table(index='children', values='debt', aggfunc= {'debt':['count','sum','mean']})

,count,mean,sum
children,,,
0,14086.0,0.075465,1063.0
1,4807.0,0.092157,443.0
2,2051.0,0.094100,193.0
3,330.0,0.081818,27.0
4,41.0,0.097561,4.0
5,9.0,0.000000,0.0


**Вывод**

По имеющимся данным мы можем сделать вывод о том, что бездетные реже имеют задолжности по кредитам, так как среди этой категории процент должников самый низкий - 7,5%. Однако, мы можем видеть, что среди людей с пятью детьми, вообще нет должников, но в этом случае мы не можем утверждать что-либо, так как в этой группе всего 9 людей - слишком мало для выводов, зависимость может быть случайной

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
df.pivot_table(index='family_status', values='debt', aggfunc= {'debt':['count','sum','mean']})

,count,mean,sum
family_status,,,
Не женат / не замужем,2795.0,0.097674,273.0
в разводе,1189.0,0.070648,84.0
вдовец / вдова,951.0,0.066246,63.0
гражданский брак,4131.0,0.093198,385.0
женат / замужем,12258.0,0.075461,925.0


**Вывод**

Задолжности по кредитам более распространы среди не женатых/не замужних или находящихся в гражданском браке людей - примерно на одном уровне - 9,8% и 9,3% должников соответственно. Менее склонны к задолжностям люди в официальном браке, либо бывшие в официальном браке (вдовцы и разведенные)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
df.pivot_table(index='income_category', values='debt', aggfunc= {'debt':['count','sum','mean']})

,count,mean,sum
income_category,,,
богатые,5310.0,0.070056,372.0
ниже среднего,10114.0,0.084042,850.0
низкий доход,371.0,0.061995,23.0
самые богатые,222.0,0.063063,14.0
средний доход,5307.0,0.088751,471.0


**Вывод**

Наименее склонны к задолжностям люди с самыми низкими доходами - среди них 6,2% должников, также среди самых богатых процент должников всего 6,3%. Больше задолжностей имеют люди со средним доходом

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
df.pivot_table(index='purpose_category', values='debt', aggfunc= {'debt':['count','sum','mean']})

,count,mean,sum
purpose_category,,,
автомобиль,4278.0,0.093268,399.0
недвижимость,10746.0,0.072492,779.0
образование,3988.0,0.092528,369.0
свадьба,2312.0,0.079152,183.0


**Вывод**

Задолжностей больше, если цель кредита автомобиль или образование

## Шаг 4. Общий вывод

Итого можно сделать вывод, что наименее склонен к задолжности гражданин без детей, в официальном браке (или бывший там), с низким доходом (< 50000) и целью кредита на недвижимость. Наиболее склонен к задолжности граждонин не в браке, с 1-2 детьми, средним доходом и целью на автомобиль или образование.